In [1]:
from adapt_utils import replace_symbols

from tree_parser import *

import networkx as nx

import matplotlib
import matplotlib.pyplot as plt

import copy

import time

from adapt_utils import replace_symbols
from my_utils import print_proof_props_graph, get_proof_steps, print_ident_proof, print_proof_linear_steps
from my_utils import get_proof_steps_graph, print_proof_steps_graph

In [2]:
text = file_contents()
database = meta_math_database(text,n=4000)

included 5555695 tokens from set.mm
proposition: 4000

In [3]:
prop = database.propositions["dfss2"]

In [4]:
print_proof_linear_steps("dfss2", database)

[dfss] |- ( A ⊆ B ↔ A = ( A ∩ B ) )
[df-in] |- ( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }
[eqeq2i] |- ( A = ( A ∩ B ) ↔ A = { x | ( x ∈ A ∧ x ∈ B ) } )
[abeq2] |- ( A = { x | ( x ∈ A ∧ x ∈ B ) } ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[3bitri] |- ( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[pm4.71] |- ( ( x ∈ A → x ∈ B ) ↔ ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[albii] |- ( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[bitr4i] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [5]:
#1. Olhar pro passo
#2. Pegar as variaveis F do teorema correspondente ao passo
#3. Preencher dentro dos passos dentro da prova do teorema

In [6]:
print_proof_linear_steps("dfss2", database)

[dfss] |- ( A ⊆ B ↔ A = ( A ∩ B ) )
[df-in] |- ( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }
[eqeq2i] |- ( A = ( A ∩ B ) ↔ A = { x | ( x ∈ A ∧ x ∈ B ) } )
[abeq2] |- ( A = { x | ( x ∈ A ∧ x ∈ B ) } ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[3bitri] |- ( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[pm4.71] |- ( ( x ∈ A → x ∈ B ) ↔ ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[albii] |- ( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
[bitr4i] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [8]:
from tree_parser import proof_step

def expand_proof_step(root_step):

    #rep_dict = root_step.tree._rep_dict
    
    expanded_steps = []
    
    for child_step in root_step.prop.entails_proof_steps:
        exp_tree = child_step.tree.copy()
        
        #Check if the child step tree has a replace dict list (axioms dont have it I think)
        replace_dict_list = child_step.replace_dict_list.copy()
        #if hasattr(child_step.tree, "replace_dict_list"):
        #    exp_tree.replace_dict_list = child_step.tree.replace_dict_list.copy()
        #else:
            #exp_tree.replace_dict_list = []
        
        replace_dict_list.extend(root_step.replace_dict_list)
        
        for rep_dict in replace_dict_list:
            exp_tree.replace(rep_dict)
        
        prior_statements = child_step.prior_statements if hasattr(child_step, "prior_statements") else []
        
        exp_step = proof_step(exp_tree, root_step.context, child_step.prop, prior_statements)
        exp_step.replace_dict_list = replace_dict_list
        
        expanded_steps.append(exp_step)
        
    return expanded_steps

In [9]:
done_steps = []

In [10]:
root_step = database.propositions["dfss2"].entails_proof_steps[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)


print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[bitr4i] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

--- [bicomi] |- ( ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ↔ ∀ x ( x ∈ A → x ∈ B ) )
--- [bitri] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [11]:
root_step = exp_proof[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)


print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[bitri] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

--- [sylbb] |- ( A ⊆ B → ∀ x ( x ∈ A → x ∈ B ) )
--- [sylbbr] |- ( ∀ x ( x ∈ A → x ∈ B ) → A ⊆ B )
--- [impbii] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [12]:
root_step = exp_proof[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)


print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[impbii] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

--- [impbi] |- ( ( A ⊆ B → ∀ x ( x ∈ A → x ∈ B ) ) → ( ( ∀ x ( x ∈ A → x ∈ B ) → A ⊆ B ) → ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) ) ) )
--- [mp2] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [13]:
root_step = exp_proof[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)


print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[mp2] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

--- [ax-mp] |- ( ∀ x ( x ∈ A → x ∈ B ) → ∀ x ( x ∈ A → x ∈ B ) )


KeyError: 'wch'

In [67]:
done_steps[0].replace_dict_list

[{'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
  'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}]

In [68]:
done_steps[1].replace_dict_list

[{'wph': <Tree: wph()>, 'wps': <Tree: wps()>, 'wch': <Tree: wch()>},
 {'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
  'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}]

In [69]:
done_steps[2].replace_dict_list

[{'wph': <Tree: wph()>, 'wps': <Tree: wch()>},
 {'wph': <Tree: wph()>, 'wps': <Tree: wps()>, 'wch': <Tree: wch()>},
 {'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
  'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}]

In [72]:
done_steps[3].tree.replace_dict_list

AttributeError: 'Tree' object has no attribute 'replace_dict_list'

In [71]:
database.propositions["mp2"].tree.replace_dict_list

[{'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wss(cB(),cC())>,
  'wch': <Tree: wss(cA(),cC())>}]